# Web Scraper

In [1]:
import requests
from bs4 import BeautifulSoup 

#Scrape the source code from the specified link:
#In this example I'm using my own medium wall
data = requests.get("https://fabiom91.medium.com")
soup = BeautifulSoup(data.content, 'html5lib')

In [2]:
import re

keywords= ['privacy','data','analytics','data science','cybersecurity']
urls = []
for key in keywords:
    html = str(soup.find_all('a', attrs = {'href':re.compile(r'^.*\b%s\b.*$' % key)}))
    href_start = [s.start() for s in re.finditer('href="',html)]
    for start in href_start:
        url = html[start+6:]
        url = url[:url.find('"')]
        if not any(c in '#?^%*()=' for c in url):
            if 'http' in url:
                urls.append(url)
            else:
                urls.append("https://fabiom91.medium.com" + url)
            
urls = list(set(urls))

In [3]:
from tqdm import tqdm
previews = []
for article in tqdm(urls):
    data = requests.get(article)
    soup = BeautifulSoup(data.content, 'html5lib')
    
    title = soup.find('h1').contents[0]
    subtitle = soup.find('p').contents[0]
    subtitle = BeautifulSoup(str(subtitle), 'lxml').text
    for x in soup.find_all('img'):
        try:
            image = str(x.attrs['srcset']).split(" ")[0]
            break
        except:
            pass
        
    article_preview = {
        'title': str(title),
        'subtitle': str(subtitle),
        'image': str(image)
    }
    previews.append(article_preview)

100%|██████████| 2/2 [00:02<00:00,  1.11s/it]


In [4]:
print(previews)

[{'title': 'Reclaim your privacy by distancing from Facebook and Google', 'subtitle': 'I was only a teenager when I started using Google and Facebook. At the time I didn’t know and didn’t care much about online privacy. My first reaction to anyone who would raise to me such concern would have been', 'image': 'https://miro.medium.com/max/552/1*1rC7yeHW09wBfLx-s_O78w.jpeg'}, {'title': 'Boost your Excel productivity with Pandas', 'subtitle': 'An absolute beginners’ introduction on Python Pandas for spreadsheet data analysis for everyday use at the office. No coding experience required.', 'image': 'https://miro.medium.com/max/552/1*LuL272ejYxifY1M9osNWTA.jpeg'}]


# Newsletter template

In [5]:
template = open('email.html')
soup = BeautifulSoup(template.read(), "html.parser")

In [6]:
article_template = soup.find('div', attrs={'class':'columns'})
html_start = str(soup)[:str(soup).find(str(article_template))]
html_end = str(soup)[str(soup).find(str(article_template))+len(str(article_template)):]
html_start = html_start.replace('\n','')
html_end = html_end.replace('\n','')

In [7]:
newsletter_content = ""
for i,article in enumerate(previews):
    
    try:
        img = article_template.img
        img['src'] = article['image']
        article_template.img.replace_with(img)
    except:
        pass
    
    title = article_template.h1
    title.string = article['title'][:300]
    
    subtitle = article_template.p
    subtitle.string = article['subtitle'][:300] + "..."
    
    link = article_template.a
    link['href'] = urls[i]
    link.string = urls[i]
    article_template.a.replace_with(link)
    
    
    newsletter_content += str(article_template).replace('\n','')

email_content = html_start + newsletter_content + html_end

In [8]:
print(BeautifulSoup(email_content).prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://cdn.jsdelivr.net/npm/bulma@0.9.1/css/bulma.min.css" rel="stylesheet"/>
 </head>
 <body>
  <section class="section">
   <div class="columns">
    <div class="column is-one-fifth">
     <img alt="article_pic" src="https://miro.medium.com/max/552/1*1rC7yeHW09wBfLx-s_O78w.jpeg"/>
    </div>
    <div class="column">
     <h1 class="title">
      Reclaim your privacy by distancing from Facebook and Google
     </h1>
     <p class="subtitle">
      I was only a teenager when I started using Google and Facebook. At the time I didn’t know and didn’t care much about online privacy. My first reaction to anyone who would raise to me such concern would have been...
     </p>
     <a href="https://fabiom91.medium.com/reclaim-your-privacy-by-distancing-from-facebook-and-google-691006136b5">
      https://fabiom91.medium.com/reclaim-your-privacy-by-dist

# Send email (SMTP)

In [9]:
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [10]:
# sender_email = "my_sender_email@gmail.com"
# receiver_email = "myemail@yahoo.com"
# # sender email password
# password = "**************"

# message = MIMEMultipart("alternative")
# message["Subject"] = "My awesome newsletter"
# message["From"] = sender_email
# message["To"] = receiver_email

# # Create the plain-text and HTML version of your message
# text = "Hi, I've found some article that you might find interesting: %s" % previews
# html = email_content

# # Turn these into plain/html MIMEText objects
# part1 = MIMEText(text, "plain")
# part2 = MIMEText(html, "html")

# # Add HTML/plain-text parts to MIMEMultipart message
# # The email client will try to render the last part first
# message.attach(part1)
# message.attach(part2)

# # Create secure connection with server and send email
# context = ssl.create_default_context()
# with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
#     server.ehlo()
#     server.login(sender_email, password)
#     server.sendmail(
#         sender_email, receiver_email, message.as_string()
#     )